In [2]:
from data import augmentations
from data import dataset
from data import snapshot
from model import model,score

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from statistics import mean
from torch.autograd import Variable
import time
from PIL import Image
import random
from skimage import io, transform
from collections import OrderedDict
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split,Subset
import torchvision
from torch import nn
from PIL import Image, ImageFilter
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from matplotlib import pyplot as plt
from skimage import io, transform
import cv2
import zipfile

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [3]:
# data = zipfile.ZipFile('rep.zip', 'r')
# data.extractall('Set')

In [4]:
data = pd.read_csv('Set/kaggle_3m/data.csv')
data.head()
csv_path =   'Set/kaggle_3m/data.csv'
data_folder = 'Set/lgg-mri-segmentation/kaggle_3m'
eg_path = 'Set/lgg-mri-segmentation/kaggle_3m/TCGA_HT_8113_19930809'
eg_img = 'Set/lgg-mri-segmentation/kaggle_3m/TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_10.tif'
eg_mask = 'Set/lgg-mri-segmentation/kaggle_3m/TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_10_mask.tif'

# AUGMENTATIONS

In [6]:
data = dataset.MySet(data_folder)
print('Length of dataset is {}'. format(data.__len__()))
print('sample data: ')
print(data.__len__())
for image, mask in data:
    print(image.shape)
    print(mask.shape)
    break

Length of dataset is 3929
sample data: 
3929
torch.Size([3, 256, 256])
torch.Size([1, 256, 256])


In [7]:
trainset, valset = random_split(data, [3600, 329])

train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=10,shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=valset, batch_size=10)

In [8]:
model = model.Unet256((3,256,256)).to(device)

In [9]:
criterion = score.DiceBCELoss()


learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
epochs = 25


train_loss = []
val_loss = []
PATH = '/model_data'
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch + 1, epochs))
    start_time = time.time()
     

    
    running_train_loss = []
    
    for image,mask in train_loader: 
            image = image.to(device,dtype=torch.float)
            mask = mask.to(device,dtype=torch.float)
            
            pred_mask = model.forward(image) # forward propogation
            loss = criterion(pred_mask,mask)
            optimizer.zero_grad() # setting gradient to zero
            loss.backward()
            optimizer.step()
            running_train_loss.append(loss.item())
                              

    else:           
        running_val_loss = []
        
        with torch.no_grad():
            for image,mask in val_loader:
                    image = image.to(device,dtype=torch.float)
                    mask = mask.to(device,dtype=torch.float)                            
                    pred_mask = model.forward(image)
                    loss = criterion(pred_mask,mask)
                    running_val_loss.append(loss.item())
                    
                                    
    torch.save(model.state_dict(),PATH)
    epoch_train_loss = np.mean(running_train_loss) 
    print('Train loss: {}'.format(epoch_train_loss))                       
    train_loss.append(epoch_train_loss)
    
    epoch_val_loss = np.mean(running_val_loss)
    print('Validation loss: {}'.format(epoch_val_loss))                                
    val_loss.append(epoch_val_loss)
                      
    time_elapsed = time.time() - start_time
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

Epoch 1/25


c:\users\79514\appdata\local\programs\python\python36\lib\site-packages\torch\nn\functional.py:2952: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
c:\users\79514\appdata\local\programs\python\python36\lib\site-packages\torch\nn\functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
c:\users\79514\appdata\local\programs\python\python36\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: 

In [ ]:
plt.plot(train_loss,label='train_loss')
plt.plot(val_loss,label='val_loss')
plt.legend()
plt.title('Loss Plot')
plt.show()

In [10]:
model.state_dict()

OrderedDict([('down1.block.0.conv.weight',
              tensor([[[[-4.7205e-03,  1.1987e-01,  1.6345e-01],
                        [-1.2016e-01,  1.0793e-01,  5.1420e-02],
                        [-3.5862e-03, -1.3456e-01,  5.7088e-02]],
              
                       [[ 1.3969e-01, -1.6519e-01, -1.7774e-01],
                        [-1.8919e-01,  4.4468e-02, -5.5298e-02],
                        [-5.9780e-02,  1.7920e-01, -4.9087e-02]],
              
                       [[-1.2221e-01,  1.7311e-02, -1.0563e-01],
                        [-5.6387e-02,  1.0166e-01,  4.4416e-02],
                        [-1.3206e-01,  1.8322e-01,  1.3020e-01]]],
              
              
                      [[[-1.3119e-01,  8.3828e-02,  7.9081e-02],
                        [-1.3809e-01, -1.3525e-01, -1.5935e-01],
                        [ 8.5727e-02,  3.3530e-02,  8.1333e-02]],
              
                       [[-8.4529e-02,  1.4471e-01,  2.4574e-02],
                        [-1.5356